In [2]:
import pandas as pd

# Replace with your file path
file_path = r'D:\Projects\new\BPGscript\ExtractedMapping\test\top10k_matches_streamed.csv'

# Read only the first 5 rows
df = pd.read_csv(file_path, nrows=5)

print(df)

   Top10kIndex  BIN_top10k PCN_top10k  GRP_top10k MatchLevel  \
0            0      4336.0    MEDDADV         NaN        GRP   
1            0      4336.0    MEDDADV         NaN        GRP   
2            0      4336.0    MEDDADV         NaN        GRP   
3            0      4336.0    MEDDADV         NaN        GRP   
4            0      4336.0    MEDDADV         NaN        GRP   

                                Plan Name/Group Name Payer Name  \
0          United Healthcare Employer and Individual    OptumRx   
1  United Healthcare Employer and Individual – Co...    OptumRx   
2                                       UHC Exchange    OptumRx   
3                                                NaN    OptumRx   
4                                                NaN    OptumRx   

   Payer Parent Name Processor Name  BIN_extracted  ... Document Name  \
0                NaN        OptumRx         610279  ...    pdf_10.pdf   
1                NaN        OptumRx         610279  ...    pdf_10.

In [ ]:
import pandas as pd
from collections import defaultdict

input_csv = 'top10k_matches_streamed.csv'
output_summary = 'top10k_summary.csv'

# Store match count and level per Top10kIndex
summary = defaultdict(lambda: {
    'BIN_top10k': '',
    'PCN_top10k': '',
    'GRP_top10k': '',
    'MatchLevel': '',
    'NumMatches': 0
})

# Stream CSV in chunks
chunk_size = 100_000
for chunk in pd.read_csv(input_csv, chunksize=chunk_size):
    for _, row in chunk.iterrows():
        idx = row['Top10kIndex']
        level = row['MatchLevel']

        # Initialize or update summary
        if summary[idx]['NumMatches'] == 0:
            summary[idx]['BIN_top10k'] = row.get('BIN_top10k', '')
            summary[idx]['PCN_top10k'] = row.get('PCN_top10k', '')
            summary[idx]['GRP_top10k'] = row.get('GRP_top10k', '')
            summary[idx]['MatchLevel'] = level

        # Increment match count unless "No match found"
        if level != 'No match found':
            summary[idx]['NumMatches'] += 1

# Convert summary to DataFrame and sort by index (to keep top10k order)
summary_df = pd.DataFrame([
    {
        'Top10kIndex': idx,
        'BIN_top10k': data['BIN_top10k'],
        'PCN_top10k': data['PCN_top10k'],
        'GRP_top10k': data['GRP_top10k'],
        'MatchLevel': data['MatchLevel'],
        'NumMatches': data['NumMatches']
    }
    for idx, data in sorted(summary.items())
])

# Save to CSV
summary_df.to_csv(output_summary, index=False)
print(f"✅ Summary saved to: {output_summary}")


In [1]:
import pandas as pd
 
file_path = "55final_5level_joins_with_skipped.xlsx"
final_df = pd.read_excel(file_path,sheet_name="Sheet1", dtype=str)
final_df.fillna("(NULL)", inplace=True)
 
# Desired column order
desired_columns = [
    "BIN_top10k", "PCN_top10k", "GRP_top10k", "PLAN_ID", "PLAN_NAME", "ROW_CNT", "RATIO", "BPG_top10k",
    "Full_BPG_key", "Bin_Grp_key", "Grp_key", "Bin_PCN_key", "Bin_key",
    "FULLBPGValid", "Bin_GrpValid", "GRPValid", "Bin_PCNValid", "BINValid", "ALLBlank",
    "Plan Name/Group Name", "Payer Name", "Payer Parent Name", "Processor Name",
    "BIN_extracted", "PCN_extracted", "GRP_extracted", "Effective Date", "Document Name",
    "Beneficiary Type", "Address", "Phone Number", "Page Number",
    "Matched Payer Parents", "Matched Payer Names", "Matched Processor Name", "Comments", "BPG_extracted",
    "Bin_Grp_key_right", "Grp_key_right", "Bin_PCN_key_right", "Bin_key_right",
    "FULLBPGValid_right", "Bin_GrpValid_right", "GRPValid_right", "Bin_PCNValid_right", "BINValid_right", "ALLBlank_right",
    "Join_Level", "Matched", "Full_BPG_key_right"
]
 
for col in desired_columns:
    if col not in final_df.columns:
        final_df[col] = "(NULL)"
 
final_df = final_df[desired_columns]
 
final_df.to_excel("final_final_df_reordered.xlsx", index=False)